# Integration Method by Matt (Example using `IndShockConsumer`)

Constructing End of Period (Marginal) Value Function using Transitionmatrices instead of Expectations.
This notebook shows the necessary steps to construct the function. For simplicity, we will only focus on transitory shock without unemployment probability nor permanent shocks.

Idea by Matt White
Code by Adrian Monninger

In [ ]:
from HARK.interpolation import LinearInterp, MargValueFuncCRRA
from HARK.ConsumptionSaving.ConsIndShockModel import IndShockConsumerType
import numpy as np

from HARK.distribution import (
    expected,
)

import matplotlib.pyplot as plt

from HARK.utilities import make_grid_exp_mult
import scipy as sp

In [ ]:
T_cycle = 5
PermShk = 0.0
TranShk = 0.2
ShockCount = 21
UnempPrb = 1e-10
CRRA = 2
Rfree = 1.04**0.25
DiscFac = 0.975
PermGroFac = 1.0
LivPrb = 0.99375
DiscFacEff = DiscFac * LivPrb

Dict = {
    "CRRA": CRRA,
    "Rfree": [Rfree] * T_cycle,  # Interest factor on assets
    "DiscFac": DiscFac,  # Intertemporal discount factor
    "LivPrb": [LivPrb]
    * T_cycle,  # [0.999], #[.99375],                  # Survival probability
    "PermGroFac": [PermGroFac] * T_cycle,  # Permanent income growth factor
    # Parameters that specify the income distribution over the lifecycle
    "PermShkStd": [PermShk]
    * T_cycle,  # [.06],                 # Standard deviation of log permanent shocks to income
    "PermShkCount": ShockCount,  # Number of points in discrete approximation to permanent income shocks
    "TranShkStd": [TranShk]
    * T_cycle,  # [.2],                   # Standard deviation of log transitory shocks to income
    "TranShkCount": ShockCount,  # Number of points in discrete approximation to transitory income shocks
    "UnempPrb": [UnempPrb] * T_cycle,  # Probability of unemployment while working
    "IncUnemp": [0.0] * T_cycle,  # Unemployment benefits replacement rate
    "UnempPrbRet": [0.0] * T_cycle,  # Probability of "unemployment" while retired
    "IncUnempRet": [0.0] * T_cycle,  # "Unemployment" benefits when retired
    "T_cycle": T_cycle,  # Number of periods in the cycle for this agent type
}

In [ ]:
IndShock = IndShockConsumerType(**Dict)
IndShock.update()
IndShock.solve()

In [ ]:
t = 0
tNext = 1

In [ ]:
### Get Last Period Marginal Value Function and grids
# cFuncNext = IndShock.solution[1].cFunc
# vPfuncNext = MargValueFuncCRRA(cFuncNext, CRRA)
vPfuncNext = IndShock.solution[tNext].vPfunc
aXtraGrid = IndShock.aXtraGrid
mNrmMinNext = IndShock.solution[tNext].mNrmMin

### Shocks
IncShkDstn = IndShock.IncShkDstn[t]
ShkPrbsNext = IncShkDstn.pmv
PermShkValsNext = IncShkDstn.atoms[0]
TranShkValsNext = IncShkDstn.atoms[1]
PermShkMinNext = np.min(PermShkValsNext)
TranShkMinNext = np.min(TranShkValsNext)

# Borrowing Constraint
BoroCnstNat = (
    (IndShock.solution[tNext].mNrmMin - TranShkMinNext)
    * (IndShock.PermGroFac[t] * PermShkMinNext)
    / IndShock.Rfree[t]
)

aNrmNow = np.asarray(aXtraGrid) + BoroCnstNat

In [ ]:
def m_nrm_next(PermGroFac, shocks, a_nrm, Rfree):
    """
    Computes normalized market resources of the next period
    from income shocks and current normalized market resources.

    Parameters
    ----------
    shocks: [float]
        Permanent and transitory income shock levels.
    a_nrm: float
        Normalized market assets this period

    Returns
    -------
    float
        normalized market resources in the next period
    """
    return Rfree / (PermGroFac * shocks["PermShk"]) * a_nrm + shocks["TranShk"]

In [ ]:
### End-of-period Marginal Value Function (Traditional Way)


def vp_next(shocks, a_nrm, Rfree):
    return shocks["PermShk"] ** (-CRRA) * vPfuncNext(
        m_nrm_next(PermGroFac, shocks, a_nrm, Rfree)
    )


EndOfPrdvP = (
    DiscFacEff
    * Rfree
    * PermGroFac ** (-CRRA)
    * expected(vp_next, IncShkDstn, args=(aNrmNow, Rfree))
)

In [ ]:
plt.plot(aNrmNow, EndOfPrdvP)
plt.title("End Of Period Marginal Value Function")
plt.ylim([0.0, 2.0])
plt.show()

### Alternative Start

In [ ]:
### Step 1: In a pre-solution step, specify fairly dense grids of m and b, denser than the grid of a.
mGrid = make_grid_exp_mult(0.0001, 52, 310, timestonest=1)
bGrid = make_grid_exp_mult(0.0, 48, 300, timestonest=1)

### Let ist start from minimum of aNrm
# NO, WRONG MINIMUM!
mGrid = mGrid + mNrmMinNext
bGrid = bGrid + aNrmNow[0]

In [ ]:
plt.plot(mGrid, vPfuncNext(mGrid))
plt.ylim([0.0, 2.0])
plt.title("Next period marginal value function")
plt.show()

In [ ]:
### Step 2: In that pre-solution step, for each $F_{\theta, t}$ (i.e. just once if this is an ifinite horizon problem),
###         calculate $f_{\theta, t}(m - b) on the cross product of the b-grid and m-grid.
###         Put them in a matrix that has b values by row and m values by column.
###         Still in the pre-solution step, for each b-row in the matrix, take the row-wise sum and
###         divide the row by it. This represents a Markov transition matrix from the exogenous
###         b-grid to the exogenous m-grid.


def updateBM_TranMatrix(TranShkStd, bGrid, mGrid):
    """
    Calculates the Probabillity of a transioty shock for the b times m matrix
    """
    # probGrid = np.zeros((len(self.bNrmGrid_income), len(self.mNrmGrid_income)))  # b x m
    # ### getting the probability for each transitory shock with the size b - m (remember m = b * transitory shock)

    ### Integration 1: No unemployment probability
    s = TranShkStd
    mu = -0.5 * s**2
    lognorm_dist = sp.stats.lognorm(s, scale=np.exp(mu))

    ### Create matrix
    # Construct meshgrid of bNrmGrid_income and mNrmGrid_income
    b, m = np.meshgrid(bGrid, mGrid, indexing="ij")

    # Calculate differences between corresponding elements
    probGrid = lognorm_dist.pdf(m - b)

    for i_b in range(len(bGrid)):
        probGrid[i_b] = probGrid[i_b] / (np.max([np.sum(probGrid[i_b]), 0.000001]))

    return probGrid

In [ ]:
probGrid = updateBM_TranMatrix(TranShk, bGrid, mGrid)

In [ ]:
probGrid.shape

In [ ]:
### Step 3: To compute expectations during backwards solution, evaluate vt(mt) on the dense m-grid. Put
### these (marginal) values into a matrix that has one column.
vPnext_array = vPfuncNext(mGrid)

In [ ]:
plt.plot(mGrid, vPnext_array)
plt.title("Next period marginal value function")
plt.xlabel("m Next grid")
plt.ylim([0.0, 2.0])
plt.show()

In [ ]:
### Step 4: Pre-multiply the Markov matrix by the matrix you just constructed. The resulting
###         matrix of intermediate expected values will have b row-wise.

vPnext_array_reshaped = np.reshape(vPnext_array, (vPnext_array.size, 1))
Interm_vP = np.dot(probGrid, vPnext_array_reshaped)
Interm_vPnvrs = Interm_vP.flatten() ** (-1 / CRRA)
Interm_vPnvrsFunc = LinearInterp(
    np.insert(bGrid, 0, 0.0), np.insert(Interm_vPnvrs, 0, 0.0)
)  # IMPORTANT: GENERALIZE THIS ZERO
Interm_vPfunc = MargValueFuncCRRA(Interm_vPnvrsFunc, CRRA)

In [ ]:
plt.plot(bGrid, Interm_vPfunc(bGrid))
plt.title("Intermediate Marginal Value Function")
plt.xlabel("b Next Grid")
plt.ylim([0.0, 2.0])
plt.show()

In [ ]:
### Step 5: Using the pre-specified exogenous grid of a, compute expected end-of-period
###         (marginal) value using the typical discretized approximation to the permanent shock
###         distribution. For any (a) end-of-period state, there is a finite set of future (b)
###         points that can be reached under the discretization.

In [ ]:
def b_nrm_next(PermGroFac, shocks, a_nrm, Rfree):
    """
    Computes normalized bank balances of the next period
    from income shocks and current normalized market resources.

    Parameters
    ----------
    shocks: [float]
        Permanent and transitory income shock levels.
    a_nrm: float
        Normalized market assets this period

    Returns
    -------
    float
        normalized market resources in the next period
    """
    return Rfree / (PermGroFac * shocks["PermShk"]) * a_nrm

In [ ]:
def vp_next_Integration(shocks, a_nrm, Rfree):
    return shocks["PermShk"] ** (-CRRA) * Interm_vPfunc(
        b_nrm_next(PermGroFac, shocks, a_nrm, Rfree)
    )

In [ ]:
EndOfPrdvP_Integration = (
    DiscFacEff
    * Rfree
    * PermGroFac ** (-CRRA)
    * expected(vp_next_Integration, IncShkDstn, args=(aNrmNow, Rfree))
)

In [ ]:
plt.plot(aNrmNow, EndOfPrdvP)
plt.plot(aNrmNow, EndOfPrdvP_Integration, "--")
plt.title("End Of Period Marginal Value Functions: Comparison")
plt.xlabel("a Grid")
plt.ylim([0.0, 2.0])
plt.show()

In [ ]:
# Construct the consumption function in two ways: standard approach and new approach
cNrm_old = EndOfPrdvP ** (-1 / CRRA)
mNrm_old = aNrmNow + cNrm_old
cFunc_old = LinearInterp(np.insert(mNrm_old, 0, 0.0), np.insert(cNrm_old, 0, 0.0))

cNrm_new = EndOfPrdvP_Integration ** (-1 / CRRA)
mNrm_new = aNrmNow + cNrm_new
cFunc_new = LinearInterp(np.insert(mNrm_new, 0, 0.0), np.insert(cNrm_new, 0, 0.0))

In [ ]:
# Plot a comparison of the two consumption functions using different integration methods
M = np.linspace(0.0, 10.0, 201)
C0 = cFunc_old(M)
C1 = cFunc_new(M)
plt.plot(M, C0)
plt.plot(M, C1)
plt.xlabel(r"Market resources $m_t$")
plt.ylabel(r"Consumption $c_t$")
plt.xlim(0.0, 10.0)
plt.ylim(0.0, None)
plt.title("Consumption function comparison")
plt.show()

In [ ]:
plt.plot(M, C1 - C0)
plt.xlabel(r"Market resources $m_t$")
plt.ylabel(r"Difference between methods")
plt.title("Consumption function comparison")
plt.xlim(0.0, 10.0)
plt.show()